## Creating Dataset for Machine Learning with SD Score

The below script creates a dataset for machine learning with the following features:

- TVL: Total Value Locked in USD
- APY: Annual Percentage Yield
- APY Mean 7D: The average APY over the last 7 days
- APY Std 7D: The standard deviation of the APY over the last 7 days
- TVL Percentile: The percentile of TVL at each date
- APY 7D Percentile: The percentile of the 7 day APY at each date
- APY 30D Percentile: The percentile of the 30 day APY at each date
- APY 7D Std Ratio: The average APY for 7 days divided by the standard deviation of the APY for 7 days
- TVL Change 7D: The change in TVL over the last 7 days
- TVL Change 1D: The change in TVL over the last 1 day
- SD Score: The product of '7 day APY percentile' and 'tvl percentile' multiplied by 100
- SD Score 7D Avg: The rolling mean of the past 7 days SD score
- SD Score 7D Std: The standard deviation of the SD_Score_7D over the last 7 days

The dataset is a derivtive and statiscial description of TVL and APY data for DeFi protocols. The source of the data is DeFiLlama



In [1]:
# Import libraries and dependencies
import pandas as pd

data = pd.read_csv(r'/Users/karolk/Python_Work/Data_Sets/Global_Data/DeFi_Global_DB.csv', index_col=0)
pd.set_option("display.max.columns", None)
pd.set_option("display.precision", 4)
pd.options.display.float_format = '{:,.2f}'.format


/var/folders/gn/b1hbt5mx3mg2y8p3tgg7g7xh0000gn/T/ipykernel_8777/2874500894.py:4: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r'/Users/karolk/Python_Work/Data_Sets/Global_Data/DeFi_Global_DB.csv', index_col=0)


In [2]:
#set date and time added as datetime objects
data['date'] = pd.to_datetime(data['date'])
data['time added'] = pd.to_datetime(data['time added'])

#drop all pools that have a TVL of 0 or a APY of 0
data = data[(data['tvlUsd'] > 0) & (data['apy'] > 0)]

# specify the start and end date
analysis_date = pd.to_datetime('2023-12-31') # convert the end_date to a datetime object
# end date is 21 days after the analysis date
end_date = analysis_date + pd.DateOffset(days=10) 
# start date is 21 days before the analysis date
start_date = analysis_date - pd.DateOffset(days=21)
data = data[(data['date'] >= start_date) & (data['date'] <= end_date)]


In [3]:

# select only stablecoin is true
#data = data[data['stablecoin'] == True]

# sort the data table by pool and by date
data = data.sort_values(['pool', 'date'], ascending=[True, True]).reset_index(drop=True)

# create a new column which would the average APY over the last 7 days using the 'apy' column
data['apyMean7d'] = data.groupby('pool')['apy'].transform(lambda x: x.rolling(7, 1).mean())

# create a new column for the standard deviation of the APY over the last 7 days
data['apyStd7d'] = data.groupby('pool')['apy'].transform(lambda x: x.rolling(7, 1).std())

# getting the forward mean APY for the next 7 days. 
data['apyMean7dForward'] = data.groupby('pool')['apy'].transform(lambda x: x.shift(-7).rolling(7, 1).mean())

# getting the APY in 7 days
data['apy7dForward'] = data.groupby('pool')['apy'].shift(-7)

# creating a column for change in APY over the last 7 days measured
data['apyChange7d'] = data.groupby(['pool'])['apy'].pct_change(7)*100

# creating a column for the change in APY over the last 7 days in percentage
data['apyChange7dPercent'] = data.groupby(['pool'])['apy'].pct_change(7) * 100

# create a column which has the percentile for TVL for the TVL at each date
data['tvlPercentile'] = data.groupby('date')['tvlUsd'].rank(pct=True)

# create a column which has the percentile for 7 day APY
data['apy7DPercentile'] = data.groupby('date')['apyMean7d'].rank(pct=True)

# creating 2 new columns which is the average APY for 7 days divided by the standard deviation of the APY for 7 days
data['apy7DStdRatio'] = data['apyMean7d'] / data['apyStd7d']

# calculate a new column for the change in TVL over the last 7 days
data['tvlChange7d'] = data.groupby(['pool'])['tvlUsd'].pct_change(periods=7) * 100

# creating column with 'SD_Score' which is the product of '7 day APY percentile' and 'tvl percentile' multiplied by 100
data['SD_Score'] = data['apy7DPercentile'] * data['tvlPercentile'] * 100

# creating a column 'SD_Score_7D' which is the rolling mean of the past 7 days SD score
data['SD_Score_7D_avg'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.rolling(7, 1).mean())

# create a column 'SD_Score_7D_std' which is the standard deviation of the SD_Score_7D over the last 7 days
data['SD_Score_7D_std'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.rolling(7, 1).std())

# create a new column for forward SD_Score_7D_avg
data['SD_Score_7D_forward_rolling'] = data.groupby('pool')['SD_Score'].transform(lambda x: x.shift(-7).rolling(7, 1).mean())

# create a new column for the forward SD_Score_7D which is the SD_Score 7 days in the future
data['SD_Score_7D_forward'] = data.groupby('pool')['SD_Score'].shift(-7)

# create a new column which is the change in SD_Score over the last 7 days
data['SD_Score_7D_change'] = data.groupby(['pool'])['SD_Score'].pct_change(7)*100

# create a new column which is the change in SD_Score over the last 7 days
data['SD_Score_7D_forward_change'] = data.groupby(['pool'])['SD_Score'].pct_change(-7)*100


In [4]:
# Data Transformation - Adding new columns for the underlying tokens & taking only the most recent date

# create a list of all unique symbols in the data set
symbols = data['symbol'].unique()

# sort the symbols alphabetically
symbols.sort()

# splitting the 'symbol' column into 4 new columns using str.split() method with '-' as the separator
symbol_split = data['symbol'].str.split('-', expand=True, n=3)

# adding the 4 new columns to the data dataframe
symbol_split.columns = ['token_id_1', 'token_id_2', 'token_id_3', 'token_id_4']
data = pd.concat([data, symbol_split], axis=1)

# create a new column called num_tokens which is the number of tokens in the symbol
data['num_tokens'] = data['symbol'].str.count('-') + 1


In [5]:
# select the data that is equal to or less than the analysis date
data = data[data['date'] <= analysis_date]

# select data that is 7 days after the start date
data = data[data['date'] >= start_date + pd.DateOffset(days=7)]

# select data that is 7 days before the end date

In [6]:
data

,chain,project,symbol,tvlUsd,apy,pool,stablecoin,ilRisk,exposure,outlier,apyMean30d,date,time added,new_upload,possible_error,apyMean7d,apyStd7d,apyMean7dForward,apy7dForward,apyChange7d,apyChange7dPercent,tvlPercentile,apy7DPercentile,apy7DStdRatio,tvlChange7d,SD_Score,SD_Score_7D_avg,SD_Score_7D_std,SD_Score_7D_forward_rolling,SD_Score_7D_forward,SD_Score_7D_change,SD_Score_7D_forward_change,token_id_1,token_id_2,token_id_3,token_id_4,num_tokens
0,MultiversX,xexchange,UTK-WEGLD,"2,536,115.00",16.17,d044b46e-b525-4a70-b1b0-7bea71c0c5a2,False,yes,multi,True,17.46,2023-12-27,2023-12-27 07:01:31,False,False,16.17,NaN,NaN,NaN,NaN,NaN,0.91,0.58,NaN,NaN,52.66,52.66,NaN,NaN,NaN,NaN,NaN,UTK,WEGLD,None,None,2
1,Ethereum,yearn-finance,LINK,"2,074,346.00",0.84,d04eaa00-199c-40c8-b72d-16570fd1bd12,False,no,single,False,0.70,2023-12-27,2023-12-27 07:01:31,False,False,0.84,NaN,NaN,NaN,NaN,NaN,0.90,0.16,NaN,NaN,14.06,14.06,NaN,NaN,NaN,NaN,NaN,LINK,None,None,None,1
2,Solana,raydium,UNKNOWN-SOL,"14,907.00","14,004.22",d05678c9-f8a4-4150-885e-3f0529b838b7,False,yes,multi,True,"3,709.79",2023-12-20,2023-12-20 07:31:38,False,False,"14,004.22",NaN,NaN,NaN,NaN,NaN,0.42,1.00,NaN,NaN,42.11,42.11,NaN,NaN,NaN,NaN,NaN,UNKNOWN,SOL,None,None,2
3,Solana,raydium,SOL-UNKNOWN,"10,421.00","9,148.57",d05ad1e8-20a2-4d9b-b1d4-f8e898b517c0,False,yes,multi,True,"18,079.45",2023-12-27,2023-12-27 07:01:31,False,False,"9,148.57",NaN,NaN,NaN,NaN,NaN,0.03,0.98,NaN,NaN,2.78,2.78,NaN,NaN,NaN,NaN,NaN,SOL,UNKNOWN,None,None,2
4,Ethereum,convex-finance,DOLA-FRAXBP,"12,535,230.00",23.44,d05cb04d-f1e5-451d-95a2-6a3a9da001ad,True,no,multi,False,17.73,2023-12-27,2023-12-27 07:01:31,False,False,23.44,NaN,NaN,NaN,NaN,NaN,0.98,0.64,NaN,NaN,63.03,63.03,NaN,NaN,NaN,NaN,NaN,DOLA,FRAXBP,None,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,Ethereum,convex-finance,PBTC-SBTCCRV,"361,151.00",0.01,ffd0679c-1a42-414e-ac73-83c01d1ba764,False,no,multi,False,0.12,2023-12-27,2023-12-27 07:01:31,False,False,0.01,NaN,NaN,NaN,NaN,NaN,0.71,0.02,NaN,NaN,1.45,1.45,NaN,NaN,NaN,NaN,NaN,PBTC,SBTCCRV,None,None,2
2475,Arbitrum,flashstake,SGLP,"207,179.00",17.64,ffd1ae24-1fdc-4ae5-a0a7-44d74b63962f,False,no,single,False,16.83,2023-12-27,2023-12-27 07:01:31,False,False,17.64,NaN,NaN,NaN,NaN,NaN,0.63,0.59,NaN,NaN,37.34,37.34,NaN,NaN,NaN,NaN,NaN,SGLP,None,None,None,1
2476,Tezos,matter-defi,BTCTZ(ALT)-WTZ,"17,251.00",0.14,ffd531d7-152b-4a38-a509-cc4cce09dee0,False,yes,multi,False,0.15,2023-12-27,2023-12-27 07:01:31,False,False,0.14,NaN,NaN,NaN,NaN,NaN,0.18,0.07,NaN,NaN,1.24,1.24,NaN,NaN,NaN,NaN,NaN,BTCTZ(ALT),WTZ,None,None,2
2478,Arbitrum,beefy,DOGE-USDC,"15,446.00",45.23,ffee9154-b34a-4241-a42f-9fec2a92ba2b,False,yes,multi,True,60.51,2023-12-27,2023-12-27 07:01:31,False,False,45.23,NaN,NaN,NaN,NaN,NaN,0.15,0.75,NaN,NaN,11.50,11.50,NaN,NaN,NaN,NaN,NaN,DOGE,USDC,None,None,2


In [7]:

columns = data.columns.tolist()

# create a list of columns that we want to keep

columns_to_keep = ['date', 'pool', 'symbol', 'tvlUsd', 'apy', 'apyMean7d', 'apyStd7d', 'apyMean7dForward', 'apy7dForward', 'apyChange7d', 'apyChange7dPercent', 'tvlPercentile', 'apy7DPercentile', 'apy7DStdRatio', 'tvlChange7d', 'SD_Score', 'SD_Score_7D_avg', 'SD_Score_7D_std', 'SD_Score_7D_forward_rolling', 'SD_Score_7D_forward', 'SD_Score_7D_change', 'SD_Score_7D_forward_change', 'token_id_1', 'token_id_2', 'token_id_3', 'token_id_4', 'num_tokens']

# select only the columns that we want to keep

data = data[columns_to_keep]

In [8]:

data

,date,pool,symbol,tvlUsd,apy,apyMean7d,apyStd7d,apyMean7dForward,apy7dForward,apyChange7d,apyChange7dPercent,tvlPercentile,apy7DPercentile,apy7DStdRatio,tvlChange7d,SD_Score,SD_Score_7D_avg,SD_Score_7D_std,SD_Score_7D_forward_rolling,SD_Score_7D_forward,SD_Score_7D_change,SD_Score_7D_forward_change,token_id_1,token_id_2,token_id_3,token_id_4,num_tokens
0,2023-12-27,d044b46e-b525-4a70-b1b0-7bea71c0c5a2,UTK-WEGLD,"2,536,115.00",16.17,16.17,NaN,NaN,NaN,NaN,NaN,0.91,0.58,NaN,NaN,52.66,52.66,NaN,NaN,NaN,NaN,NaN,UTK,WEGLD,None,None,2
1,2023-12-27,d04eaa00-199c-40c8-b72d-16570fd1bd12,LINK,"2,074,346.00",0.84,0.84,NaN,NaN,NaN,NaN,NaN,0.90,0.16,NaN,NaN,14.06,14.06,NaN,NaN,NaN,NaN,NaN,LINK,None,None,None,1
2,2023-12-20,d05678c9-f8a4-4150-885e-3f0529b838b7,UNKNOWN-SOL,"14,907.00","14,004.22","14,004.22",NaN,NaN,NaN,NaN,NaN,0.42,1.00,NaN,NaN,42.11,42.11,NaN,NaN,NaN,NaN,NaN,UNKNOWN,SOL,None,None,2
3,2023-12-27,d05ad1e8-20a2-4d9b-b1d4-f8e898b517c0,SOL-UNKNOWN,"10,421.00","9,148.57","9,148.57",NaN,NaN,NaN,NaN,NaN,0.03,0.98,NaN,NaN,2.78,2.78,NaN,NaN,NaN,NaN,NaN,SOL,UNKNOWN,None,None,2
4,2023-12-27,d05cb04d-f1e5-451d-95a2-6a3a9da001ad,DOLA-FRAXBP,"12,535,230.00",23.44,23.44,NaN,NaN,NaN,NaN,NaN,0.98,0.64,NaN,NaN,63.03,63.03,NaN,NaN,NaN,NaN,NaN,DOLA,FRAXBP,None,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,2023-12-27,ffd0679c-1a42-414e-ac73-83c01d1ba764,PBTC-SBTCCRV,"361,151.00",0.01,0.01,NaN,NaN,NaN,NaN,NaN,0.71,0.02,NaN,NaN,1.45,1.45,NaN,NaN,NaN,NaN,NaN,PBTC,SBTCCRV,None,None,2
2475,2023-12-27,ffd1ae24-1fdc-4ae5-a0a7-44d74b63962f,SGLP,"207,179.00",17.64,17.64,NaN,NaN,NaN,NaN,NaN,0.63,0.59,NaN,NaN,37.34,37.34,NaN,NaN,NaN,NaN,NaN,SGLP,None,None,None,1
2476,2023-12-27,ffd531d7-152b-4a38-a509-cc4cce09dee0,BTCTZ(ALT)-WTZ,"17,251.00",0.14,0.14,NaN,NaN,NaN,NaN,NaN,0.18,0.07,NaN,NaN,1.24,1.24,NaN,NaN,NaN,NaN,NaN,BTCTZ(ALT),WTZ,None,None,2
2478,2023-12-27,ffee9154-b34a-4241-a42f-9fec2a92ba2b,DOGE-USDC,"15,446.00",45.23,45.23,NaN,NaN,NaN,NaN,NaN,0.15,0.75,NaN,NaN,11.50,11.50,NaN,NaN,NaN,NaN,NaN,DOGE,USDC,None,None,2


In [9]:
# save the data to a csv file
filepath= 'Datasets/DeFi_Quant_Data.csv'
data.to_csv(filepath, index=False)